In [ ]:
##### PTT 股票版 

### 取得首頁 source code
import requests
from bs4 import BeautifulSoup as bs 

url = "https://www.ptt.cc/bbs/Stock/index.html"
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res = requests.get(url,headers=headers)
soup = bs(res.text,"lxml")

In [ ]:
### 抓取首頁文章連結
links = []
for a_tag in soup.select("div#main-container div.r-ent div.title a"):
    
    # 過濾 版規 & 盤後閒聊 / 盤中閒聊
    title = a_tag.text
    
    if "股票板板規" in title or "盤後閒聊" in title or "盤中閒聊" in title :
        continue # 跳過此步, 執行下一動迴圈
    else:
        url = "https://www.ptt.cc" + a_tag["href"]
        print(url,title)

In [ ]:
### 抓取 分頁文章 連結

headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

for i in range(1,6):
    
    # 建構 '上頁' 連結
    link = soup.select("div#action-bar-container div.btn-group-paging a")[1]["href"]
    previous_link = "https://www.ptt.cc" + link

    res = requests.get(previous_link,headers=headers)
    soup = bs(res.text,"lxml")

    for a_tag in soup.select("div#main-container div.r-ent div.title a"):

        # 過濾 版規 & 盤後閒聊 / 盤中閒聊
        title = a_tag.text

        if "股票板板規" in title or "盤後閒聊" in title or "盤中閒聊" in title :
            continue # 跳過此步, 執行下一動迴圈
        else:
            url = "https://www.ptt.cc" + a_tag["href"]
            links.append(url)
            
    print("{} is ok.".format(previous_link))
        
print("Done.")

In [ ]:
### 抓取文章本文 source code 
url = links[11]
print(url)

headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res2 = requests.get(url,headers=headers)
soup2 = bs(res2.text,"lxml")

In [ ]:
### 抓取本文的 作者 , 看板 , 標題 , 時間 
span_tags = soup2.select("div#main-content span.article-meta-value")

# 作者
author = span_tags[0].text
print("author :",author)

# 看板
category = span_tags[1].text
print("category :",category)

# 標題
title = span_tags[2].text
print("title :",title)

# 時間
time = span_tags[3].text
print("time :",time)

# span_tags

In [ ]:
### 抓取本文的 內容 , 回應
# 標籤拔除 --> .extract()
# 為了標籤拔除 , 故先抓取 回應資料

# push_tags = soup2.select("div#main-content div.push")

# 抓其中一項做測試
# ele = push_tags[77]
# span_tags = ele.select("span")

# resp = {
#     "tag"     : span_tags[0].text.strip(),
#     "author"  : span_tags[1].text.strip(),
#     "content" : span_tags[2].text.replace(": ","").strip(), 
#     "time"    : span_tags[3].text.strip()
# }
# print(resp)
# span_tags

#############
# for-loop 處理全部回應
# resp_data = []

# for ele in push_tags:
#     span_tags = ele.select("span")

#     resp = {
#         "tag"     : span_tags[0].text.strip(),
#         "author"  : span_tags[1].text.strip(),
#         "content" : span_tags[2].text.replace(": ","").strip(), 
#         "time"    : span_tags[3].text.strip()
#     }
    
#     resp_data.append(resp)
    
#############
# 加入拔除標籤動作
def get_resp_data(ele):
    span_tags = ele.select("span")
    return {
        "tag"     : span_tags[0].text.strip(),
        "author"  : span_tags[1].text.strip(),
        "content" : span_tags[2].text.replace(": ","").strip(), 
        "time"    : span_tags[3].text.strip()
    }


push_tags = soup2.select("div#main-content div.push")
resp_data = []

if len(push_tags) >0:
    
    for ele in push_tags:
        ele.extract()  # 宣告從 div#main-content 中,拔除 div.push 標籤
        
        resp = get_resp_data(ele)
        
        resp_data.append(resp)
        
#         span_tags = ele.select("span")
        
#         resp = {
#             "tag"     : span_tags[0].text.strip(),
#             "author"  : span_tags[1].text.strip(),
#             "content" : span_tags[2].text.replace(": ","").strip(), 
#             "time"    : span_tags[3].text.strip()
#         }

    
# print(resp_data)

In [ ]:
### 內容
# print(soup2.select("div#main-content")[0].text)

# 1. 移除標籤
# - div.article-metaline
# - div.article-metaline-right
# - span.f2

def remove_dirty_tag(soup):
    
    # 若存在 , 則移除標籤
    if len(soup.select("div.article-metaline")) >0 :
        
        # 標籤可能多項 , 使用 for-loop 移除
        for tag in soup.select("div.article-metaline"):
            tag.extract()
            
    if len(soup.select("div.article-metaline-right")) >0 :
        for tag in soup.select("div.article-metaline-right"):
            tag.extract()
            
    if len(soup.select("span.f2")) >0 :
        for tag in soup.select("span.f2"):
            tag.extract()
    
    return soup 



soup2 = remove_dirty_tag(soup2)

# 2. 抓取文字
content = soup2.select("div#main-content")[0].text.strip()
print("content:", content)